In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define the function

In [2]:
def function(x):
    return np.exp(-2*x) * np.cos(10*x)

In [14]:
def function_integral(x):
    return (np.exp(-2*x))*(5*np.sin(10*x) - np.cos(10*x))/52

## Define the Trapezoid Core

In [5]:
def trapezoid_core(f, x, h):
    return 0.5*h*(f(x+h) + f(x))

## Define Wrapper Function for Trapezoid Method

In [6]:
def trapezoid_method(f, a, b, N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evaluations to use
    
    #x values to perform trapezoid rule
    x = np.linspace(a, b, N)
    h = x[1] - x[0]
    
    #the value of the integral
    Fint = 0.0
    
    #perform the integral
    for i in range(0, len(x) - 1, 1):
        Fint += trapezoid_core(f, x[i], h)
        
    #the answer
    return Fint

## Define the Simpson's Method

In [7]:
def simpson_core(f, x, h):
    return h*(f(x) + 4*f(x+h) + f(x+2*h)) / 3.0

## Wrapper for Simpson's Method

In [8]:
def simpsons_method(f, a, b, N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evaluations to use
    # note the number of chunks will be N - 1
    # so if N is odd, then we don't need to adjust the last segment
    
    #x values to perform simpsons rule
    x = np.linspace(a, b, N)
    h = x[1] - x[0]
    
    #the value of the integral
    Fint = 0.0
    
    #perform the integral
    for i in range(0, len(x) - 2, 2):
        Fint += simpson_core(f, x[i], h)
        
    #apply simpson's rule over the last interval if N is even
    if((N % 2) == 0):
        Fint += simpson_core(f, x[-2], 0.5*h)
        
    return Fint

## Romberg Integration

In [9]:
def romberg_core(f, a, b, i):
    
    #the difference b - a
    h = b - a
    
    #the increment between new func evals
    dh = h / 2.**(i)
    
    #the cofactor
    K = h / 2.**(i+1)
    
    #the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #the answer
    return K * M

## Romberg Wrapper Function

In [12]:
def romberg_integration(f, a, b, tol):
    
    #define an iteration variable
    i = 0 
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error estimate, set to a large value
    delta = 100.0 * np.fabs(tol)
    
    #set an array if integral answers
    I = np.zeros(imax, dtype = float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta > tol):
        
        #find this romberg iteration
        I[i] = 0.5 * I[i-1] + romberg_core(f, a, b, i)
        
        #compute the new fractional error estimate
        delta = np.fabs( (I[i] - I[i-1]) / I[i] )
        
#        print(i, I[i], I[i-1], delta)
        
        if(delta > tol):
            
            #iterate 
            i += 1
            
            #if we've reached the max iterations
            if(i > imax):
                print("Max iterations reached.")
                raise StopIteration("Stopping iterations after ", i)
                
    print('Romberg Integration done with', i, 'iterations')
    return I[i]

## Confirm Answer

In [15]:
answer = function_integral(np.pi) - function_integral(0)
answer

0.019194856870544078

## Romberg Integration

In [16]:
tolerance = 1.0e-6
a = 0
b = np.pi
RI = romberg_integration(function, a, b, tolerance)
print(RI)


Romberg Integration done with 26 iterations
0.019194869607236313


## Trapezoid Method

In [18]:
print(trapezoid_method(function, a, b, 4000))

0.01919495953857451


### Took 4000 evaluations for T.M. to reach specified accuracy

## Simpson's Method

In [20]:
print(simpsons_method(function, a, b, 500))

0.01919485170855182


In [21]:
## Took 500 evalutations for S.M. to reach specified accuracy